In [1]:
import sys
import os
import PIL
from PIL import Image
import psutil
import time

sys.path.append(os.path.abspath("../"))
from src.helper import rgb_to_cmyk, calculate_cmyk_percentage, predict_pdf, label_to_price

In [2]:
initial_cpu = psutil.cpu_percent(interval=None)
initial_memory = psutil.virtual_memory().used

img = Image.open("../outputs/pdfium_10dpi/page-001.jpg").convert("RGB")
final_cpu = psutil.cpu_percent(interval=None)
final_memory = psutil.virtual_memory().used

# Menghitung perubahan
cpu_usage = final_cpu - initial_cpu
memory_usage = final_memory - initial_memory

# print(f"Waktu eksekusi: {end_time - start_time:.4f} detik")
print(f"Penggunaan CPU: {cpu_usage:.2f}%")
print(f"Penggunaan memori: {memory_usage / (1024 ** 2):.2f} MB") 

Penggunaan CPU: 30.50%
Penggunaan memori: 0.77 MB


In [3]:
start = time.time()
output = predict_pdf("../datasets/statistik-indonesia-2024.pdf")
print(time.time() - start)

False True


/Users/rasyidev/miniconda/envs/print-cost/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/rasyidev/miniconda/envs/print-cost/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


page-1: 3
False True
page-2: 2
False True
page-3: 0
False True
page-4: 2


/Users/rasyidev/miniconda/envs/print-cost/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/rasyidev/miniconda/envs/print-cost/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/rasyidev/miniconda/envs/print-cost/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/rasyidev/miniconda/envs/print-cost/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/rasyidev/miniconda/envs/print-cost/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fit

False True
page-5: 2
False True
page-6: 2
False True
page-7: 2
False True
page-8: 4
False True
page-9: 5
False True
page-10: 2
False True
page-11: 2
False True
page-12: 2
False True
page-13: 2
False True
page-14: 2
False True
page-15: 2
False True
page-16: 2


...

/Users/rasyidev/miniconda/envs/print-cost/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/rasyidev/miniconda/envs/print-cost/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/rasyidev/miniconda/envs/print-cost/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/rasyidev/miniconda/envs/print-cost/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/rasyidev/miniconda/envs/print-cost/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fit

False True
page-830: 0
False True
page-831: 0
False True
page-832: 0
False True
page-833: 0
False True
page-834: 4
False True
page-835: 4
False True
page-836: 0
False True
page-837: 0
False True
page-838: 0
False True
page-839: 0
False True
page-840: 0
False True
page-841: 0
False True
page-842: 2
False True
page-843: 2
False True
page-844: 2
False True
page-845: 2
False True
page-846: 2
False True
page-847: 2
False True
page-848: 2
False True
page-849: 2
False True
page-850: 2
False True
page-851: 2


/Users/rasyidev/miniconda/envs/print-cost/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/rasyidev/miniconda/envs/print-cost/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/rasyidev/miniconda/envs/print-cost/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/rasyidev/miniconda/envs/print-cost/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/rasyidev/miniconda/envs/print-cost/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fit

False True
page-852: 3
38.18067502975464


/Users/rasyidev/miniconda/envs/print-cost/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [4]:
print(time.time() - start)
result = label_to_price(output)
result

38.18633484840393


{'pages_price_500': 102000,
 'pages_price_1000': 629000,
 'pages_price_2000': 38000,
 'price_total': 769000}

In [5]:
print(time.time() - start)
result = label_to_price(output, verbose=True)
result

38.20006704330444

            500  x 102000
            1000 x 629000
            2000 x 38000
        


{'pages_price_500': 102000,
 'pages_price_1000': 629000,
 'pages_price_2000': 38000,
 'price_total': 769000}

In [19]:
output

,time,label
0,0.217229,NaN
1,0.001429,NaN
2,0.165383,NaN
3,0.010500,NaN
4,0.008579,NaN
...,...,...
847,0.001125,NaN
848,0.012000,NaN
849,0.008135,NaN
850,0.001142,NaN


In [4]:
df_test = pd.read_csv("../datasets/300dpi-6-cluster-with-label.csv")
df_test

,Unnamed: 0,sum,label
0,0,157.95,3
1,1,0.07,2
2,2,15.69,0
3,3,2.99,2
4,4,4.03,2
...,...,...,...
847,847,0.07,2
848,848,7.15,2
849,849,6.86,2
850,850,0.07,2


In [14]:
output

,time,label
0,0.228686,3
1,0.001398,2
2,0.171098,0
3,0.010053,2
4,0.008653,2
...,...,...
847,0.001084,2
848,0.011503,2
849,0.008435,2
850,0.001095,2


In [15]:
true_count = 0
for pred, test in zip(output['label'], df_test['label']):
    true_count += 1

print(true_count/len(df_test) * 100)

100.0


In [9]:
df_test['label'] = df_test['label'].map({0:1000, 4:1000, 1:2000, 3:2000, 5:2000, 2:500})
df_test

,Unnamed: 0,sum,label
0,0,157.95,2000
1,1,0.07,500
2,2,15.69,1000
3,3,2.99,500
4,4,4.03,500
...,...,...,...
847,847,0.07,500
848,848,7.15,500
849,849,6.86,500
850,850,0.07,500


In [14]:
a = df_test['label'].value_counts()

In [23]:
a

label
1000    641
500     193
2000     18
Name: count, dtype: int64

In [9]:
import pandas as pd

In [10]:
df = pd.read_csv("../datasets/cmyk-by-dpi.csv")
df = df[df['dpi'] == 5]
df['sum'] = df['c'] + df['m'] + df['y'] + df['k']

In [11]:
import numpy as np
import pickle
kmeans, scaler = pickle.load(open('../models/kmeans_and_scaler.pkl', 'rb'))

In [14]:
scaled_df = scaler.transform(df['sum'].to_numpy().reshape(-1,1))
scaled_df

/Users/rasyidev/miniconda/envs/print-cost/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([[ 1.05115662e+01],
       [-1.29751582e+00],
       [-3.78078753e-02],
       [-1.06360337e+00],
       [-9.74234819e-01],
       [-1.17939392e+00],
       [-8.52227325e-01],
       [ 1.01984944e+00],
       [ 3.31312406e+00],
       [-1.29751582e+00],
       [-7.31774066e-01],
       [-7.95497726e-01],
       [-8.66992563e-01],
       [-1.29751582e+00],
       [-9.19836573e-01],
       [-8.38239204e-01],
       [-9.01962864e-01],
       [-8.27359555e-01],
       [-8.44456147e-01],
       [-8.07931610e-01],
       [-8.18811260e-01],
       [-8.31245144e-01],
       [-8.61552738e-01],
       [-8.05600257e-01],
       [-8.49118854e-01],
       [-8.36684969e-01],
       [-8.53004443e-01],
       [-8.09485846e-01],
       [-8.49118854e-01],
       [-8.09485846e-01],
       [-8.43679029e-01],
       [-8.47564618e-01],
       [-8.08708728e-01],
       [-8.61552738e-01],
       [-8.40570558e-01],
       [-7.88503666e-01],
       [-8.61552738e-01],
       [-8.52227325e-01],
       [-8.7